### Import Modules

In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

### Load Data

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 모든 가사파일을 raw_corpus 에 넣음.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


### Set Hyperparameters

In [3]:
num_words=12000
random_state= 0

embedding_size = 512
hidden_size = 1024
batch_size= 128

### Preprocessing

#### Clean data by removeing unnecessary parts

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백 제거
    sentence = re.sub('[\[]+[a-zA-Z0-9\s]+[\]]', '',sentence) ## 대괄호에 쓰여있는 instruction 제거
#     sentence = re.sub('[\(+[a-zA-Z0-9\s]+[\)]', '',sentence) ## 괄호에 쓰여있는 instruction 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백을 하나의 공백으로 변경
    sentence = re.sub(r"[^a-zA-Z0-9?.!,¿\[\]]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 변경
    sentence = sentence.strip() # 5. 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에 <start>, 끝에 <end>를 추가
    return sentence

#### remove sentences having more that 15 words

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue ## 빈 문장 제거
    preprocessed_sentence = preprocess_sentence(sentence)
    
    pre_list = preprocessed_sentence.split(' ') ## 정제 후 빈 문장 제거
    if len(pre_list) <= 3:  
        pre_list.remove('<start>')
        pre_list.remove('<end>')
        if pre_list== ['']: continue
            
#     if preprocessed_sentence in corpus: print(preprocessed_sentence) # continue ## 정제 후 이미 같은 문장이 corpus 에 있으면 추가하지 않음
    
    if len(preprocessed_sentence.split(' ')) > 17 : continue ## 17 단어보다 긴 문장 추가하지 않음 (15단어 + <start>, <end>)
        
    corpus.append(preprocessed_sentence)

corpus

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>',
 '<start> who feel that life is but a joke <end>',
 '<start> but , uh , but you and i , we ve been through that <end>',
 '<start> and this is not our fate <end>',
 '<start> so let us stop talkin falsely now <end>',
 '<start> the hour s getting late , hey all along the watchtower <end>',
 '<start> princes kept the view <end>',
 '<start> while all the women came and went <end>',
 '<start> barefoot servants , too <end>',
 '<start> outside in the cold distance <end>',
 '<start> a wildcat did growl <end>',

In [6]:
len(raw_corpus)

187088

In [7]:
len(corpus)

163203

In [8]:
# 15개 보다 많은 단어로 된 문장이 제거 되었는지 확인 (start, end 포함시에는 17로 변경하여 확인)
for sentence in corpus:
    if len(sentence.split(' ')) > 17:
        print(sentence)

#### 텍스트 데이터를 토큰화 하여 텐서로 변환

In [9]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words,  # 12000단어를 기억할 수 있는 tokenizer를 생성
        filters=' ',
        oov_token="<unk>" ## 12000단어에 포함되지 못한 단어는 '<unk>'로 변경 (unknown)
    )
    
    ## corpus를 이용해 tokenizer 내부의 단어장에 저장
    tokenizer.fit_on_texts(corpus)
    
    ## tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # padding 을 사용하여 입력 데이터의 시퀀스 길이를 일정하게 맞춤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 276 ...   0   0   0]
 [  2 113   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  76  47 ...   0   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 653 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fbc0d0f4790>


#### 위에서 만든 tensor를  source 와  target 데이터로 분리 (data and label)

In [10]:
src_input = tensor[:, :-1]  ## tensor에서 마지막 토큰 삭제
tgt_input = tensor[:, 1:]  ## tensor에서 <start>를 잘라내어 타겟 문장 생성

#### split into train & validation data

In [11]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=random_state )

##### check train&test data shapes

In [12]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (130562, 16)
Target Train: (130562, 16)


### Generate Dataset
- tf.data.Dataset 사용
- train & validation 두가지 dataset 객체를 각각 생성

In [13]:
BUFFER_SIZE = len(src_input)
steps_per_epoch = len(src_input) // batch_size

## tokenizer가 구축한 단어사전 12000개에 0:<pad>를 포함하여 12001개가 됨.
VOCAB_SIZE = tokenizer.num_words + 1   

## training dataset
dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(batch_size, drop_remainder=True)

## validation dataset
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(batch_size, drop_remainder=True)


### Build Model

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [15]:
## set optimizer and compile model
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, epochs=10, validation_data= (enc_val,dec_val), batch_size= batch_size)

Epoch 1/10
1020/1020 [==============================] - 391s 377ms/step - loss: 3.3747 - val_loss: 2.7159
Epoch 2/10
1020/1020 [==============================] - 389s 382ms/step - loss: 2.6404 - val_loss: 2.5560
Epoch 3/10
1020/1020 [==============================] - 354s 347ms/step - loss: 2.4585 - val_loss: 2.4539
Epoch 4/10
1020/1020 [==============================] - 334s 327ms/step - loss: 2.3173 - val_loss: 2.3814
Epoch 5/10
1020/1020 [==============================] - 336s 329ms/step - loss: 2.1841 - val_loss: 2.3234
Epoch 6/10
1020/1020 [==============================] - 333s 326ms/step - loss: 2.0677 - val_loss: 2.2803
Epoch 7/10
1020/1020 [==============================] - 333s 326ms/step - loss: 1.9509 - val_loss: 2.2443
Epoch 8/10
1020/1020 [==============================] - 335s 328ms/step - loss: 1.8498 - val_loss: 2.2206
Epoch 9/10
1020/1020 [==============================] - 335s 328ms/step - loss: 1.7471 - val_loss: 2.2001
Epoch 10/10
1020/1020 [=======================

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


### Validation

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    ## 입력받은 init_sentence를 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        ## 1. 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        ## 2. 예측된 값 중 가장 높은 확률인 word index 선정
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        ## 3. 2에서 예측된 word index를 문장 뒤에 추가
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        ## 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    ## tokenizer를 이용해 word index를 단어로 변환 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
generate_text(model, tokenizer, init_sentence='<start> only')

'<start> only one thing i can t do <end> '

In [22]:
generate_text(model, tokenizer, init_sentence='<start> you')

'<start> you re the only one that i know <end> '

In [23]:
generate_text(model, tokenizer, init_sentence='<start> you are')

'<start> you are the one that i need <end> '

In [24]:
generate_text(model, tokenizer, init_sentence='<start> he')

'<start> he s a walker in the rain <end> '

In [25]:
generate_text(model, tokenizer, init_sentence='<start> I')

'<start> i m the one gon hold you down <end> '

In [26]:
generate_text(model, tokenizer, init_sentence='<start> you')

'<start> you re the only one that i know <end> '

In [27]:
generate_text(model, tokenizer, init_sentence='<start> I love')

'<start> i love you , i love you <end> '

In [28]:
generate_text(model, tokenizer, init_sentence='<start> one and ')

'<start> one and two , three , four , four , five , six , seven <end> '

## 회고

### Dataset
- 49개의 가사 txt 파일을 긁어 와서 총 187,088개의 문장을 데이터를 얻었으며, 그 중 가사가 아닌 부분을 제거하여 총 163,203개 문장을 사용하였다.
- 텍스트 데이터를 tocenize 하여 각각의 단어를 tokenizer 내부의 단어장에 저장하고, 저장된 단어들은 그에 상응하는 tensor 로 변환하였다. (모든 텐서는 패딩을 사용하여 일정한 길이로 맞추었다)
- 토큰화하여 생성된 텐서에서 [-1]을 제거한 source 데이터와 [0]을 제거한 target 데이터를 각각 생성하였고, 이 데이터를 80%는 train dataset (130,562개), 20% 는 validation dataset (32,641개) 으로 나누어 각각의 데이터셋 객체를 생성하였다.


### Hyperparameters
- 단어장의 총 단어수는 12,000으로 제한되었으며, epoch 도 10으로 제한되어있다.
- train_test_split 의 random_state=[0, 7, 32] 세가지와, embedding_size = [128, 256, 512], hidden_size = [512, 1024, 2048], batch_size=[32, 64, 128, 256, 512] 를 각각 실험하였다. 이 중 가장 좋은 성능을 나타낸 hyperparameter는 result 에 추가하였다.


### Model structure
- One embedding layer
- Two rnn layers with LSTM
- One dense layer


### Analysis
- random_state=32 로 시작했을 때에는 loss가 처음엔 6 이상에서 시작하여. 첫 epoch 에서 loss 가 3.7로 내려갔다. (val_loss 2.8) randome_state = 0 으로 변경 후에는 loss 가 애초부터 낮게 시작하여 더 빨리 로스값을 낮출 수 있었다. 
- embedding size 와 hiddel size 의 선정이 어려웠는데, 예제에서 주어졌던 값에서 위아래로 테스트 해 나가면서 성능이 잘 나오는 쪽으로 추려갔다. 
- 대체적으로 batch size 가 낮을수록 loss 값이 낮아지는 경향이 보였으나 속도가 너무 느려져서 문제가 있었다. 
- 또한, Sequential 한 signal 에서는 너무 짧은 신호는 사용하지 않는데, nlp 에서도 어느정도의 의미 있는 문장이 사용되어야 하지 않을까..? 하는 생각에서 오는 의문점으로, batch size 가 너무 낮으면 문장을 배워야 하는 모델이 너무 짧은 단어 몇개만을 배울 가능성이 있어보여서, batch size 가 얼마나 작아져도 괜찮은지, 32, 64 는 어느정도의 특징을 가지고 있는지 알아봐야 하겠다. 
- input을 주면 어느정도 말이 되는 문장을 만들어 낸다. (nlp 처음 해봤는데, 귀엽고 신기한 것 같다!)


### Results
- 아래의 hyperparameter 을 사용하고, 10 epoch 을 돌렸을 때, val-loss = 2.17 이 나왔다. (2.1 정도의 loss 값이 나온 케이스가 몇가지 더 있었으나, 이 코드에서는 아래의 케이스를 선정하였다.)
    - random_state= 0
    - embedding_size = 512
    - hidden_size = 1024
    - batch_size= 128
- 모델이 생성한 몇가지 문장은 회고 위에서 볼 수 있다.